### Functional Richness

In [1]:
import pandas as pd

df = pd.read_excel('Hartford Infrastructure.xlsx', sheet_name = 'Richness')

zip_code = ['06103','06105','06106','06112','06114','06120']
infrastrucute_list = ['roadway','bus_route','railway','bike_lane','sidewalk']
infrastrucute_length = list(map(lambda x: x + '_length', infrastrucute_list))
richness_table = pd.DataFrame({'zip_code':zip_code})

#Functional richness calculation
for i in range(len(infrastrucute_list)):
    richness_table[infrastrucute_list[i]] = df[infrastrucute_length[i]] / df['area_size']
richness_table['carpool'] = df['carpoolers'] / df['area_size']

#Normalization
richness_table.iloc[:,1:] = richness_table.iloc[:,1:].apply(lambda x: x / max(x))

#Data output
richness_table.columns = ['zip_code','roadway_r','bus_r','railway_r','bike_lane_r','sidewalk_r','carpool_r']
richness_table

,zip_code,roadway_r,bus_r,railway_r,bike_lane_r,sidewalk_r,carpool_r
0,06103,1.000000,1.000000,0.536824,0.970481,1.000000,1.000000
1,06105,0.469405,0.134271,0.329867,0.899526,0.530311,0.611340
2,06106,0.517775,0.132447,0.228084,1.000000,0.613188,0.321380
3,06112,0.451487,0.045855,0.234636,0.834348,0.501268,0.154711
4,06114,0.401644,0.075388,0.334457,0.406345,0.322803,0.107527
5,06120,0.286692,0.082441,1.000000,0.582212,0.268936,0.086292


### Functional Evenness

In [2]:
import pandas as pd

zip_code = ['06103','06105','06106','06112','06114','06120']
infrastrucute_list = ['roadway','bus_stop','railway_station','bike_lane','sidewalk','carpool']
evenness_table = pd.DataFrame(columns = ['roadway_e','bus_e','railway_e','bike_lane_e','sidewalk_e','carpool_e'])

#Functional evenness calculation
for i in range(len(zip_code)):
    df = pd.read_excel('Hartford Infrastructure.xlsx', sheet_name = zip_code[i])
    reciprocal = 1 / df.shape[0]
    df.iloc[:,1:] = df.iloc[:,1:].apply(lambda x: x / x.sum())
    df.iloc[:,1:] = df.iloc[:,1:].applymap(lambda x: min(x, reciprocal))
    evenness_table.loc[i] = df.iloc[:,1:].apply(lambda x: (x.sum() - reciprocal) / (1 - reciprocal)).tolist()

#Normalization
evenness_table = evenness_table.apply(lambda x: x / max(x))

#Data output
evenness_table

,roadway_e,bus_e,railway_e,bike_lane_e,sidewalk_e,carpool_e
0,0.880845,1.000000,0.697047,0.536924,1.000000,0.988420
1,0.865657,0.771958,0.875848,0.969330,0.484341,1.000000
2,0.921043,0.750801,0.848905,0.876184,0.582225,0.739748
3,1.000000,0.833348,0.927677,1.000000,0.398976,0.836328
4,0.909988,0.778664,1.000000,0.982242,0.333791,0.863922
5,0.939762,0.837361,0.864647,0.927211,0.380599,0.810151


### Entropy Weight Method

In [3]:
import math
import pandas as pd

df = pd.concat([richness_table, evenness_table], axis=1)
row = df.shape[0]
col = df.shape[1]

#Infrastructure diversity calculation
df.iloc[:,1:] = df.iloc[:,1:].apply(lambda x: x / max(x), axis=1)
df.iloc[:,1:] = df.iloc[:,1:].apply(lambda x: x / x.sum())
df.iloc[:,1:] = df.iloc[:,1:].applymap(lambda x: -x * math.log(x) if x != 0.0 else 0.0)
E_list = df.iloc[:,1:].apply(lambda x: x.sum() / math.log(row)).tolist()
df['Diversity'] = 0
for i in range(len(E_list)):
    W = (1 - E_list[i]) / (col - 1 - sum(E_list))
    df['Diversity'] += W * pd.concat([richness_table, evenness_table], axis=1).iloc[:,i+1]

#Normalization
df[['Diversity']] = df[['Diversity']].apply(lambda x: x / max(x))

#Data output
#df[['zip_code','Diversity']].to_csv('Infrastructure Diversity.csv', index=False)
df[['zip_code','Diversity']]

,zip_code,Diversity
0,06103,1.000000
1,06105,0.387800
2,06106,0.318580
3,06112,0.214100
4,06114,0.196781
5,06120,0.264733
